In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read Data

In [22]:
# https://nycmaps-nyc.hub.arcgis.com/datasets/nyc::building/about
buildings_raw = pd.read_csv('BUILDING_view_7607496916235021567.csv')

# https://data.cityofnewyork.us/City-Government/Property-Address-Directory/bc8t-ecyu/about_data
zipcodes_raw = pd.read_csv('bobaadr.txt', low_memory=False)

# https://www.nyc.gov/site/planning/data-maps/open-data/dwn-pluto-mappluto.page
pluto_raw = pd.read_csv('pluto_25v1.csv', low_memory=False)

# https://data.cityofnewyork.us/Public-Safety/Risk-Based-Inspections-RBIS-/itd7-gx3g/about_data
inspection_raw = pd.read_csv('Risk_Based_Inspections__RBIS__20250308.csv', low_memory=False)

# https://simplemaps.com/data/us-zips
zip_area_raw = pd.read_csv('uszips.csv')

# Clean Data
### building data

In [3]:
buildings_raw.columns

Index(['OBJECTID', 'Base BBL', 'BIN', 'Construction Year', 'DOITT ID',
       'Feature Code', 'Geometry Source', 'Ground Elevation', 'Height Roof',
       'LAST_EDITED_DATE', 'LAST_STATUS_TYPE', 'Map Pluto BBL', 'Name', 'Area',
       'Length'],
      dtype='object')

In [4]:
buildings = buildings_raw[['OBJECTID', 'BIN', 'Construction Year', 'DOITT ID', 'Feature Code', 'Height Roof',
       'LAST_STATUS_TYPE','Map Pluto BBL', 'Area', 'Length']]

In [5]:
# Keep only LAST_STATUS_TYPE == "Constructed"
buildings = buildings[buildings['LAST_STATUS_TYPE'] == "Constructed"]
buildings.drop('LAST_STATUS_TYPE', axis=1, inplace=True)

### zipcode data

In [6]:
zipcodes = zipcodes_raw[['bin','zipcode']]
zipcodes.head()

,bin,zipcode
0,1000000,10004
1,1000000,10004
2,1000000,10004
3,1000000,10004
4,1000000,10004


### pluto data

In [7]:
pluto = pluto_raw[['borough', 'block', 'lot', 'bbl', 'zipcode', 'latitude', 'longitude']]
pluto.head()

,borough,block,lot,bbl,zipcode,latitude,longitude
0,BK,5852,1,3.058520e+09,11220.0,40.638298,-74.030598
1,BK,5852,13,3.058520e+09,11220.0,40.638575,-74.030126
2,BK,5852,6,3.058520e+09,11220.0,40.638567,-74.030490
3,BK,5852,58,3.058520e+09,11220.0,40.638142,-74.029704
4,BK,5848,77,3.058480e+09,11220.0,40.639039,-74.030115


In [8]:
# Convert bbl column from float to int
pluto['bbl'] = pluto['bbl'].apply(lambda x: int(x))

C:\Users\kevin\AppData\Local\Temp\ipykernel_28504\1061226196.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pluto['bbl'] = pluto['bbl'].apply(lambda x: int(x))


### inspection data

In [9]:
inspection = inspection_raw[['INSP_INSPECT_DT', 'BBL', 'BLDG_CURRENT_BIN_FK']]
inspection.head()

,INSP_INSPECT_DT,BBL,BLDG_CURRENT_BIN_FK
0,08/14/2018,4.000510e+09,4000431
1,01/29/2016,3.050480e+09,3115797
2,05/21/2014,5.024508e+09,5120386
3,05/17/2016,1.022480e+09,1065067
4,06/01/2018,3.009170e+09,3018012


### zip area data

In [35]:
zip_area = zip_area_raw[['zip', 'population', 'density']]
# Density in population per km^2 (1 km^2 = 0.386102 mile^2)
zip_area['area_mile2'] = zip_area['population'] / zip_area['density'] * 0.386102
zip_area = zip_area[['zip', 'area_mile2']]

C:\Users\kevin\AppData\Local\Temp\ipykernel_28504\2234351423.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zip_area['area_mile2'] = zip_area['population'] / zip_area['density'] * 0.386102


# Merge data
## Find zipcodes
### buildings with zipcodes & pluto

In [36]:
# Buildings with zipcodes
num_empty = buildings.merge(zipcodes, left_on='BIN', right_on='bin', how='left').value_counts('zipcode').max()
print(f'There are {num_empty} rows with empty zipcode')

There are 77488 rows with empty zipcode


In [41]:
# Buildings with pluto
data = buildings.merge(pluto, left_on='Map Pluto BBL', right_on='bbl', how='left').drop('Map Pluto BBL', axis=1)

# Merge with zip_area
data = data.merge(zip_area, left_on = 'zipcode', right_on='zip', how='left')
data.head()

,OBJECTID,BIN,Construction Year,DOITT ID,Feature Code,Height Roof,Area,Length,borough,block,lot,bbl,zipcode,latitude,longitude,zip,area_mile2
0,1,3170958,1925.0,96807,2100,29.749853,264.464844,74.281490,BK,6522.0,21.0,3.065220e+09,11230.0,40.626025,-73.966724,11230.0,1.842909
1,2,5028452,1965.0,326368,2100,22.630000,121.714844,44.163804,SI,1264.0,36.0,5.012640e+09,10303.0,40.639310,-74.167926,10303.0,3.169214
2,3,5078368,1970.0,746627,2100,35.760000,115.828125,43.920822,SI,6019.0,91.0,5.060190e+09,10312.0,40.556102,-74.195182,10312.0,6.871665
3,4,3245111,1928.0,786626,2100,37.500000,188.238281,73.872226,BK,8691.0,48.0,3.086910e+09,11235.0,40.577413,-73.961165,11235.0,2.457111
4,5,4161096,1950.0,746409,2100,18.015113,284.390625,72.749024,QN,7502.0,5.0,4.075020e+09,11364.0,40.755840,-73.754259,11364.0,2.508998


In [42]:
# Create building age column
data['age'] = data.apply(lambda x: 2025-x['Construction Year'], axis=1)

In [44]:
data.sample(5)

,OBJECTID,BIN,Construction Year,DOITT ID,Feature Code,Height Roof,Area,Length,borough,block,lot,bbl,zipcode,latitude,longitude,zip,area_mile2,age
589907,594671,3063909,1930.0,441080,2100,41.8800,276.777344,69.431778,BK,2504.0,22.0,3.025040e+09,11222.0,40.734781,-73.956779,11222.0,1.522369,95.0
405808,409323,4458686,1998.0,117457,2100,24.9800,237.578125,65.008255,QN,9133.0,12.0,4.091330e+09,11417.0,40.677697,-73.859413,11417.0,1.113494,27.0
934275,941420,3353759,1935.0,857037,5110,11.2100,51.859375,28.808340,BK,4713.0,7.0,3.047130e+09,11236.0,40.652482,-73.916606,11236.0,3.564266,90.0
738604,744399,4295362,1945.0,656765,2100,24.9672,128.464844,45.841921,QN,14177.0,46.0,4.141770e+09,11414.0,40.658812,-73.835608,11414.0,2.308578,80.0
652346,657548,4594030,1940.0,997316,5110,11.6663,63.980469,35.178178,QN,11809.0,9.0,4.118090e+09,11420.0,40.668436,-73.818429,11420.0,2.107772,85.0


## Find inspection risks

In [45]:
insp_hist = data.merge(inspection, left_on=['bbl', 'BIN'], right_on=['BBL', 'BLDG_CURRENT_BIN_FK'], how='right')\
            .drop(['OBJECTID', 'BBL', 'BLDG_CURRENT_BIN_FK', 'Area', 'Length', 'block', 'lot', 'Construction Year'], axis=1)\
            .dropna()
insp_hist['zipcode'] = insp_hist['zipcode'].apply(lambda x:int(x))
insp_hist

,BIN,DOITT ID,Feature Code,Height Roof,borough,bbl,zipcode,latitude,longitude,zip,area_mile2,age,INSP_INSPECT_DT
0,4000431.0,276863.0,2100.0,30.780000,QN,4.000510e+09,11101,40.748422,-73.950426,11101.0,2.587168,56.0,08/14/2018
1,3115797.0,225924.0,2100.0,46.150000,BK,3.050480e+09,11225,40.656287,-73.959764,11225.0,0.874838,94.0,01/29/2016
2,5120386.0,223676.0,2100.0,23.340000,SI,5.024508e+09,10314,40.578294,-74.157794,10314.0,12.996831,45.0,05/21/2014
3,1065067.0,265347.0,2100.0,76.060000,MN,1.022480e+09,10034,40.868562,-73.924958,10034.0,1.022458,98.0,05/17/2016
4,3018012.0,296696.0,2100.0,52.280000,BK,3.009170e+09,11232,40.649863,-74.004148,11232.0,1.241439,98.0,06/01/2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...
369014,1008937.0,695094.0,2100.0,16.740000,MN,1.005530e+09,10011,40.733084,-73.999109,10011.0,0.647556,91.0,10/16/2015
369016,4461146.0,448882.0,2100.0,24.368200,QN,4.084470e+09,11004,40.752172,-73.717018,11004.0,0.951068,77.0,03/30/2018
369017,4302052.0,574466.0,2100.0,36.397169,QN,4.159260e+09,11692,40.591876,-73.788262,11692.0,1.015846,94.0,06/27/2014
369018,5014391.0,139466.0,2100.0,9.930000,SI,5.005630e+09,10304,40.622728,-74.083231,10304.0,3.490795,57.0,03/07/2017


In [46]:
# Create inspection count using DOITT ID
insp_count = pd.DataFrame(insp_hist['DOITT ID'].value_counts()).reset_index()

In [47]:
# Merge data with insp_count
df = data.merge(insp_count, on='DOITT ID', how='left').rename({'count':'insp_count'}, axis=1)
df.insp_count = df.insp_count.fillna(0)

In [48]:
df.insp_count.quantile(0.94), df.insp_count.quantile(0.98)

(2.0, 3.0)

# Classify Fire Risks

- **High risks**: Less than 2% of the buildings had been **inspected more than 3 times** since 2014, and they will be classified as with high fire risk. Also, buildings **over 87 years old** will also be classified as with high fire risk (fire code was introduced to NYC in 1938).
- **Medium risks**: Approximately 4% of the buildings had been **inspected 2 or 3 times** since 2014, and they will be classified as with medium fire risk. Also, buildings **between 37 and 87 years old** will also be classified as with medium fire risk (major building code overhaul occurred in 1968).
- **Low risks**: Buildings with **1 or less inspections** and **less than 37 years old** will be classified as with the lowest fire risk (major building code overhaul occurred in 1968).

source: https://digitalcommons.njit.edu/dissertations/77/

In [49]:
def risk(insp_count, age):
    if insp_count > 3 or age > 87:
        return 'high'
    elif 2 <= insp_count <= 3 or 37 <= age <= 87:
        return 'medium'
    elif insp_count <= 1 or age < 37:
        return 'low'

In [50]:
df['risk'] = df.apply(lambda x: risk(x['insp_count'], x['age']), axis=1)
df.risk.value_counts()

risk
high      633751
medium    341046
low       105188
Name: count, dtype: int64

In [52]:
df = df.groupby(['zipcode', 'risk'], as_index=False).count()[['zipcode', 'risk', 'area_mile2', 'lot']]
df = df.rename({'lot':'count'}, axis=1)

In [54]:
df.to_csv('building_fire_risk.csv', index=False)